In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta

# Set seed for reproducibility
np.random.seed(42)

## Machine Profile Definitions

In [ ]:
# Machine profiles based on Factory I/O components
machine_profiles = {
    'conveyors': {
        'count': 14,
        'startup_surge': 2.0,  # kW
        'run_power': 1.0,      # kW
        'startup_time': 2,     # seconds
        'efficiency': 0.92
    },
    'pushers': {
        'count': 4,
        'activation_surge': 3.0,  # kW
        'hold_power': 0.5,        # kW
        'cycle_time': 1,          # second
        'efficiency': 0.88
    },
    'pick_place': {
        'count': 3,
        'peak_power': 4.0,     # kW
        'idle_power': 0.2,     # kW
        'operation_time': 3,   # seconds
        'efficiency': 0.90
    }
}

print("Machine Profiles:")
for machine_type, specs in machine_profiles.items():
    print(f"\n{machine_type.upper()}:")
    for key, val in specs.items():
        print(f"  {key}: {val}")

## Simulation Functions

In [ ]:
def simulate_conveyor_operation(duration_seconds, num_conveyors):
    """Simulate conveyor power consumption over time."""
    timesteps = np.arange(0, duration_seconds, 0.1)
    power_data = []
    
    for t in timesteps:
        # Simulate startup surges at random intervals
        startup_prob = 0.05
        if np.random.random() < startup_prob:
            surge_power = machine_profiles['conveyors']['startup_surge']
        else:
            surge_power = machine_profiles['conveyors']['run_power']
        
        total_power = surge_power * num_conveyors
        power_data.append(total_power)
    
    return timesteps, np.array(power_data)

def simulate_pusher_operation(duration_seconds, num_pushers):
    """Simulate pusher power consumption over time."""
    timesteps = np.arange(0, duration_seconds, 0.1)
    power_data = []
    
    for t in timesteps:
        # Pushers activate cyclically
        cycle_phase = (t % machine_profiles['pushers']['cycle_time']) / machine_profiles['pushers']['cycle_time']
        activation_prob = max(0, 1 - abs(cycle_phase - 0.5) * 4)  # Triangular activation profile
        
        active_pushers = int(num_pushers * activation_prob)
        surge_power = machine_profiles['pushers']['activation_surge'] * active_pushers
        hold_power = machine_profiles['pushers']['hold_power'] * (num_pushers - active_pushers)
        total_power = surge_power + hold_power
        power_data.append(total_power)
    
    return timesteps, np.array(power_data)

def simulate_robot_operation(duration_seconds, num_robots):
    """Simulate pick-place robot power consumption over time."""
    timesteps = np.arange(0, duration_seconds, 0.1)
    power_data = []
    
    for t in timesteps:
        # Robots operate in staggered patterns
        robot_powers = []
        for i in range(num_robots):
            phase_offset = i * machine_profiles['pick_place']['operation_time'] / num_robots
            cycle_phase = ((t + phase_offset) % (machine_profiles['pick_place']['operation_time'] * 2)) / (machine_profiles['pick_place']['operation_time'] * 2)
            
            if cycle_phase < 0.5:
                power = machine_profiles['pick_place']['peak_power']
            else:
                power = machine_profiles['pick_place']['idle_power']
            robot_powers.append(power)
        
        power_data.append(sum(robot_powers) + np.random.normal(0, 0.1))  # Add noise
    
    return timesteps, np.array(power_data)

print("Simulation functions defined.")

## Generate Operational Scenarios

In [ ]:
def generate_operational_scenarios(profiles, duration_seconds=3600):
    """Generate complete operational scenarios combining all machines."""
    
    # Get individual machine simulations
    conv_time, conv_power = simulate_conveyor_operation(duration_seconds, profiles['conveyors']['count'])
    push_time, push_power = simulate_pusher_operation(duration_seconds, profiles['pushers']['count'])
    robot_time, robot_power = simulate_robot_operation(duration_seconds, profiles['pick_place']['count'])
    
    # Combine into single scenario
    total_power = conv_power + push_power + robot_power
    
    # Create DataFrame
    df = pd.DataFrame({
        'timestamp': conv_time,
        'conveyor_power': conv_power,
        'pusher_power': push_power,
        'robot_power': robot_power,
        'total_power': total_power,
        'energy_consumed': np.cumsum(total_power) * 0.1 / 3600  # Convert to kWh
    })
    
    return df

# Generate scenario
scenario_df = generate_operational_scenarios(machine_profiles)
print(f"Generated scenario with {len(scenario_df)} timesteps")
print(f"\nFirst few rows:")
print(scenario_df.head(10))
print(f"\nSummary Statistics:")
print(scenario_df.describe())

## Visualization

In [ ]:
# Matplotlib visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

axes[0, 0].plot(scenario_df['timestamp'], scenario_df['conveyor_power'], label='Conveyors')
axes[0, 0].set_title('Conveyor Power Consumption')
axes[0, 0].set_xlabel('Time (s)')
axes[0, 0].set_ylabel('Power (kW)')
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(scenario_df['timestamp'], scenario_df['pusher_power'], label='Pushers', color='orange')
axes[0, 1].set_title('Pusher Power Consumption')
axes[0, 1].set_xlabel('Time (s)')
axes[0, 1].set_ylabel('Power (kW)')
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].plot(scenario_df['timestamp'], scenario_df['robot_power'], label='Robots', color='green')
axes[1, 0].set_title('Robot Power Consumption')
axes[1, 0].set_xlabel('Time (s)')
axes[1, 0].set_ylabel('Power (kW)')
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(scenario_df['timestamp'], scenario_df['total_power'], label='Total Power', color='red', linewidth=2)
axes[1, 1].fill_between(scenario_df['timestamp'], scenario_df['total_power'], alpha=0.3, color='red')
axes[1, 1].set_title('Total Factory Power Consumption')
axes[1, 1].set_xlabel('Time (s)')
axes[1, 1].set_ylabel('Power (kW)')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Total energy consumed: {scenario_df['energy_consumed'].iloc[-1]:.4f} kWh")

## Plotly Interactive Visualization

In [ ]:
# Plotly interactive plot
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=scenario_df['timestamp'],
    y=scenario_df['conveyor_power'],
    name='Conveyors',
    mode='lines',
    stackgroup='one'
))

fig.add_trace(go.Scatter(
    x=scenario_df['timestamp'],
    y=scenario_df['pusher_power'],
    name='Pushers',
    mode='lines',
    stackgroup='one'
))

fig.add_trace(go.Scatter(
    x=scenario_df['timestamp'],
    y=scenario_df['robot_power'],
    name='Robots',
    mode='lines',
    stackgroup='one'
))

fig.update_layout(
    title='Factory I/O Power Consumption - Stacked View',
    xaxis_title='Time (seconds)',
    yaxis_title='Power (kW)',
    hovermode='x unified',
    height=500
)

fig.show()

## Save Simulated Data

In [ ]:
# Save the simulated data to CSV
import os

output_path = os.path.join('..', 'data', 'simulated_factory_data.csv')
os.makedirs(os.path.dirname(output_path), exist_ok=True)
scenario_df.to_csv(output_path, index=False)
print(f"Data saved to {output_path}")